In [7]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [8]:
# Read csv (include humidity and temperature)
df = pd.read_csv("data.csv") 
df['time'] = pd.to_datetime(df['time'])
df = df.rename(columns={'time':'ds', 'temperature_2m (°C)': 'y'})

In [9]:
model_temp = Prophet()
model_temp.fit(df)
future = model_temp.make_future_dataframe(periods=45, freq='H')

19:44:26 - cmdstanpy - INFO - Chain [1] start processing
19:44:27 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [ ]:
forecast = model_temp.predict(future)
future_forecast = forecast[forecast['ds'] > df['ds'].max()]
future_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
3168,2025-05-13 00:00:00,25.696687,24.037654,27.447744
3169,2025-05-13 01:00:00,27.332110,25.680854,28.833419
3170,2025-05-13 02:00:00,29.347279,27.674843,30.942364
3171,2025-05-13 03:00:00,31.222378,29.682554,32.863463
3172,2025-05-13 04:00:00,32.632383,31.026456,34.179334
3173,2025-05-13 05:00:00,33.553097,32.071157,35.174876
3174,2025-05-13 06:00:00,34.098163,32.453041,35.689659
3175,2025-05-13 07:00:00,34.284649,32.723473,35.877821
3176,2025-05-13 08:00:00,33.976196,32.447623,35.559913
3177,2025-05-13 09:00:00,33.056570,31.331681,34.585358


In [11]:
# Extract predicted values after the original data
predicted_df = future_forecast[['ds', 'yhat']].rename(columns={'yhat': 'temperature_2m (°C)'})
predicted_df['source'] = 'predicted'

# Format datetime to match original format
predicted_df['ds'] = predicted_df['ds'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Original data — rename and format
original_df = df.rename(columns={'y': 'temperature_2m (°C)'})
original_df['source'] = 'actual'
original_df['ds'] = original_df['ds'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Combine actual and predicted
combined_df = pd.concat([original_df[['ds', 'temperature_2m (°C)', 'source']],
                         predicted_df[['ds', 'temperature_2m (°C)', 'source']]],
                        ignore_index=True)

# Sort by datetime as string (since both are formatted the same)
combined_df = combined_df.sort_values('ds')

# Save to CSV
combined_df.to_csv("data_with_predictions.csv", index=False)

In [12]:
with open("humid_model.pkl","wb") as file:
    pickle.dump(model_temp, file)